<a href="https://colab.research.google.com/github/murillocosta/colab-storage/blob/main/tratamentoDeDados_CDS-ERA5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#montando google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#imports
import os
import xarray as xr
import pandas as pd
import numpy as np
from google.colab import autoviz

In [ ]:
#selecionando pasta
folder_dir = "/content/drive/MyDrive/sensoriamento remoto/trampo cesinha/reanalysis"
os.chdir(folder_dir)

In [ ]:
#selecionando arquivos dentro da pasta
arquivos = os.listdir()
arquivos

In [ ]:
#abrindo cada dataset e adicionando-os a uma lista
datasets = []

for arquivo in arquivos:
    arquivo = xr.open_dataset('/content/drive/MyDrive/sensoriamento remoto/trampo cesinha/reanalysis/'+ arquivo)
    datasets.append(arquivo)

In [ ]:
#convertendo cada dataset em dataframe, retirando os dados faltantes e adicionando-os a uma lista
lista_sem_nulos = []

for dataframe in datasets:
    df = dataframe.to_dataframe().reset_index()
    df_sem_nulos = df.dropna()
    lista_sem_nulos.append(df_sem_nulos)

In [ ]:
lista_sem_nulos[0].columns

In [ ]:
#dropando coluna expver do primeiro dataset
lista_sem_nulos[0] = lista_sem_nulos[0].drop('expver', axis=1)

In [ ]:
#retirando as colunas de longitude e latitude,
#adicionando cada dataframe corrigido em um lista
lista_sem_nulos_corrigida = []

for dataframe in lista_sem_nulos:
    dataframe = dataframe.drop(['longitude', 'latitude'], axis=1)
    lista_sem_nulos_corrigida.append(dataframe)

In [ ]:
#unindo dos dataframes corrigos em um só dataframe
dataframe_final = pd.concat(lista_sem_nulos_corrigida)

In [ ]:
#enviando dataframe final em formato csv para o drive
dataframe_final.to_csv('/content/drive/MyDrive/sensoriamento remoto/trampo cesinha/reanalysis/dataframe_final.csv')

In [ ]:
dataframe_final.describe()

In [ ]:
dataframe_final.info()

In [ ]:
dataframe_final.head()

In [ ]:
# Define tempo('time') como o índice do DataFrame
dataframe_final.set_index('time', inplace=True)

In [ ]:
# Calcula a média diária para cada coluna
df_media_diaria = dataframe_final.resample('D').mean()
df_media_diaria

In [ ]:
# Calcula a média diária para cada coluna
df_media_semanal = dataframe_final.resample('W').mean()
df_media_semanal

In [ ]:
def scatter_plots(df, colname_pairs, figscale=1, alpha=.8):
  from matplotlib import pyplot as plt
  plt.figure(figsize=(len(colname_pairs) * 6 * figscale, 6 * figscale))
  for plot_i, (x_colname, y_colname) in enumerate(colname_pairs, start=1):
    ax = plt.subplot(1, len(colname_pairs), plot_i)
    df.plot(kind='scatter', x=x_colname, y=y_colname, s=(32 * figscale), alpha=alpha, ax=ax)
    ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = scatter_plots(df_media_semanal, *[[['mdts', 'mdww'], ['mdww', 'mpts'], ['mpts', 'mpww'], ['mpww', 'mwd']]], **{})
chart

In [ ]:
def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(df_media_semanal, *['mdts'], **{})
chart

In [ ]:
def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(df_media_semanal, *['mdts'], **{})
chart

In [ ]:
#envia resultado das medias diarias e semanais para o google drive
df_media_semanal.to_csv('/content/drive/MyDrive/sensoriamento remoto/trampo cesinha/reanalysis/df_media_semanal.csv')
df_media_diaria.to_csv('/content/drive/MyDrive/sensoriamento remoto/trampo cesinha/reanalysis/df_media_diaria.csv')